# Lab 6 - Query feature extraction with OpenAI "Function Calling"

In [ ]:
require 'json'
require 'openai'
require 'dotenv'
require 'date'

Dotenv.load

$client = OpenAI::Client.new(access_token: ENV['OPENAI_API_KEY'])

In [ ]:
$completion_cache = {}

def get_gpt_completion(context)
  return $completion_cache[context] if $completion_cache.key?(context)
  
  messages = [{ role: "user", content: context }]
  functions = [
    {
      "name" => "extract_property_search_values",
      "description" => "For a property search system, we need to extract intent based values from a user query.",
      "parameters" => {
        "type" => "object",
        "properties" => {
          "bedrooms" => {
            "type" => "integer",
            "description" => "The number of bedrooms"
          },
          "square_footage" => {
            "type" => "integer",
            "description" => "Total square footage"
          },
          "features" => {
            "type" => "string",
            "description" => "Any additional features or architectural styles"
          },
          "non_property_related" => {
            "type" => "boolean",
            "description" => "Flag to indicate if the query is not related to property search"
          }
        },
        "required" => ["bedrooms", "square_footage", "features", "non_property_related"]
      }
    }
  ]

  response = $client.chat(
    parameters: {
      model: "gpt-3.5-turbo",
      temperature: 0,
      messages: messages,
      functions: functions,
      function_call: "auto"
    }
  )
  
  $completion_cache[context] = response
end

In [ ]:
def extract_search_values(querystring)
  gpt_res = get_gpt_completion(querystring)
  
  if gpt_res
    response_message = gpt_res.dig("choices", 0, "message")
    
    # Check if the model wants to call a function
    if function_call = response_message&.dig("function_call")
      # Call the function if it's the right one
      function_name = function_call["name"]
      if function_name == "extract_property_search_values"
        function_args = JSON.parse(function_call["arguments"])
        return function_args
      end
    end
  end
end

In [ ]:
extract_search_values("Nice colonial house with 3 or more bedrooms and a pool and white picket fence")

In [ ]:
extract_search_values("Who is Mr. T?")